In [51]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator

In [52]:
# params

IM_SIZE = 150
batch_size = 32

train_dir = 'train'
test_dir = 'test'

In [53]:
train_data_generator = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    horizontal_flip=True,
    zoom_range=0.2
)

test_data_generator = ImageDataGenerator(
    rescale=1.0 / 255
)

In [54]:
train_data = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(IM_SIZE, IM_SIZE),
    batch_size=batch_size
)

test_data = test_data_generator.flow_from_directory(
    test_dir,
    target_size=(IM_SIZE, IM_SIZE)
)

Found 4242 images belonging to 2 classes.


Found 400 images belonging to 2 classes.


In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, CategoricalCrossentropy

In [56]:
import random

In [57]:
random.seed(0)
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(IM_SIZE, IM_SIZE, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [58]:
"""model = Sequential([
    Conv2D(filters=16, activation='relu', kernel_size=(2, 2), input_shape=(IM_SIZE, IM_SIZE, 3)),
    Conv2D(filters=16, activation='relu', kernel_size=(2, 2)),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(filters=32, activation='relu', kernel_size=(2, 2)),
    Conv2D(filters=32, activation='relu', kernel_size=(2, 2)),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),
    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),
    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),
    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),
    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),
    MaxPool2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='sigmoid'),

])"""

"model = Sequential([\n    Conv2D(filters=16, activation='relu', kernel_size=(2, 2), input_shape=(IM_SIZE, IM_SIZE, 3)),\n    Conv2D(filters=16, activation='relu', kernel_size=(2, 2)),\n    MaxPool2D(pool_size=(2, 2)),\n\n    Conv2D(filters=32, activation='relu', kernel_size=(2, 2)),\n    Conv2D(filters=32, activation='relu', kernel_size=(2, 2)),\n    MaxPool2D(pool_size=(2, 2)),\n\n    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),\n    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),\n    Conv2D(filters=64, activation='relu', kernel_size=(2, 2)),\n    MaxPool2D(pool_size=(2, 2)),\n\n    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),\n    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),\n    Conv2D(filters=128, activation='relu', kernel_size=(2, 2)),\n    MaxPool2D(pool_size=(2, 2)),\n\n    Flatten(),\n\n    Dense(512, activation='relu'),\n    Dense(128, activation='relu'),\n    Dense(2, activation='sigmoid'),\n\n])"

In [59]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 148, 148, 128)     3584      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 74, 74, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 72, 72, 64)        73792     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 34, 34, 32)        18464     
                                                                 
 flatten_4 (Flatten)         (None, 36992)            

In [60]:
model.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics='accuracy'
)

In [64]:
model.fit(
    train_data,
    epochs=20,
    verbose=2
)

Epoch 1/20
133/133 - 51s - loss: 0.6107 - accuracy: 0.6634 - 51s/epoch - 386ms/step
Epoch 2/20
133/133 - 51s - loss: 0.6123 - accuracy: 0.6561 - 51s/epoch - 383ms/step
Epoch 3/20
133/133 - 51s - loss: 0.6109 - accuracy: 0.6638 - 51s/epoch - 385ms/step
Epoch 4/20
133/133 - 51s - loss: 0.6135 - accuracy: 0.6629 - 51s/epoch - 387ms/step
Epoch 5/20
133/133 - 51s - loss: 0.6034 - accuracy: 0.6749 - 51s/epoch - 386ms/step
Epoch 6/20
133/133 - 53s - loss: 0.6080 - accuracy: 0.6648 - 53s/epoch - 398ms/step
Epoch 7/20
133/133 - 51s - loss: 0.6052 - accuracy: 0.6697 - 51s/epoch - 387ms/step
Epoch 8/20
133/133 - 51s - loss: 0.6046 - accuracy: 0.6671 - 51s/epoch - 385ms/step
Epoch 9/20
133/133 - 51s - loss: 0.6070 - accuracy: 0.6726 - 51s/epoch - 385ms/step
Epoch 10/20
133/133 - 51s - loss: 0.6062 - accuracy: 0.6702 - 51s/epoch - 385ms/step
Epoch 11/20
133/133 - 51s - loss: 0.6014 - accuracy: 0.6780 - 51s/epoch - 385ms/step
Epoch 12/20
133/133 - 51s - loss: 0.6024 - accuracy: 0.6756 - 51s/epoch - 

In [65]:
model.evaluate(test_data)

13/13 [==============================] - 1s 104ms/step - loss: 0.7149 - accuracy: 0.5600


[0.7148718237876892, 0.5600000023841858]